# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 3, 10, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:11<00:00,  9.58s/it]


# Check local statistics

In [3]:
stat = pd.read_pickle('opt_PriceChange_buy_5m_1h.pkl')
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
2,PriceChange,3,100.00,80.00,80.00,80.00,60.00,60.00,60.00,60.00,60.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,2.72,1.29,1.06,0.78,0.25,3.92,3.65,3.65,4.01,4.52,3.04,2.81,2.23,1.86,0.83,1.52,1.80,2.03,2.17,3.78,4.19,3.29,2.94,2.61,5,76.666667,2.539583,33.333333,12.697917
1,PriceChange,2,100.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,2.00,0.54,2.25,2.87,2.70,3.20,3.08,4.60,5.73,5.73,4.77,4.55,2.38,2.45,3.34,3.61,3.12,3.30,2.91,2.00,1.40,1.75,2.28,2.90,2,52.083333,3.060833,-35.833333,6.121667
0,PriceChange,1,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,1.76,1.40,3.35,4.26,3.85,3.22,4.21,5.90,6.92,6.80,5.84,5.55,3.56,3.39,5.11,4.79,5.14,4.57,3.93,2.66,2.59,3.12,3.53,3.67,2,50.000000,4.130000,-40.000000,8.260000
5,PriceChange,6,100.00,70.00,70.00,60.00,70.00,50.00,60.00,70.00,60.00,70.00,70.00,70.00,70.00,70.00,60.00,60.00,60.00,60.00,60.00,50.00,50.00,60.00,70.00,70.00,0.95,0.50,0.81,0.86,0.78,0.51,0.63,1.49,1.38,1.91,1.65,2.00,1.68,1.21,0.65,0.65,1.07,0.99,0.37,0.19,-0.02,0.27,1.30,1.15,10,65.000000,0.957500,-50.000000,9.575000
3,PriceChange,4,100.00,85.71,71.43,57.14,57.14,42.86,57.14,42.86,42.86,57.14,57.14,57.14,57.14,57.14,42.86,42.86,57.14,57.14,57.14,57.14,57.14,57.14,57.14,57.14,0.73,1.45,0.50,0.50,0.25,0.00,0.43,0.00,-0.27,0.37,0.37,1.82,1.60,0.27,-0.55,-0.36,0.32,0.59,0.37,0.74,0.25,0.12,1.99,2.32,7,57.736667,0.575417,-85.843333,4.027917
6,PriceChange,7,90.00,60.00,60.00,60.00,70.00,50.00,60.00,60.00,60.00,60.00,70.00,70.00,70.00,70.00,50.00,50.00,50.00,60.00,50.00,50.00,50.00,60.00,60.00,60.00,0.83,0.30,0.32,0.34,0.26,0.19,0.30,1.49,1.15,1.49,1.13,2.00,1.34,1.04,0.19,0.19,0.37,0.67,0.09,0.10,-0.08,0.27,1.30,1.15,10,60.416667,0.684583,-95.833333,6.845833
4,PriceChange,5,100.00,62.50,62.50,50.00,62.50,37.50,50.00,62.50,50.00,62.50,62.50,62.50,62.50,62.50,50.00,50.00,50.00,50.00,50.00,50.00,50.00,62.50,62.50,62.50,0.83,0.30,0.32,0.21,0.17,-0.04,0.07,1.49,0.96,1.49,1.13,2.00,1.34,1.04,-0.17,-0.57,-0.30,0.10,-0.20,0.10,-0.08,0.65,2.21,2.22,8,57.812500,0.636250,-97.500000,5.090000
8,PriceChange,9,100.00,69.23,69.23,61.54,69.23,53.85,53.85,69.23,61.54,69.23,69.23,69.23,69.23,69.23,53.85,53.85,53.85,46.15,53.85,53.85,53.85,53.85,46.15,46.15,0.84,0.84,1.06,0.50,0.54,0.78,0.51,1.15,1.49,0.95,0.84,1.89,1.44,0.63,0.06,0.21,0.24,-0.11,0.11,0.63,0.25,0.12,-0.11,-0.11,13,61.218750,0.614583,-114.156250,7.989583
9,PriceChange,10,94.12,76.47,70.59,58.82,70.59,58.82,58.82,64.71,58.82,64.71,64.71,70.59,70.59,70.59,58.82,58.82,52.94,47.06,58.82,52.94,52.94,58.82,47.06,52.94,0.87,0.84,1.06,0.63,1.60,1.51,1.08,1.89,1.58,0.95,0.84,1.39,2.19,1.86,0.74,0.74,0.24,-0.11,0.37,0.56,0.25,0.24,-0.06,1.18,17,62.254583,0.935000,-131.672083,15.895000
7,PriceChange,8,10

# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,48.911181,0.698576,-44.928403,0.749826,33
2,53.224514,1.224826,-96.531319,0.555521,60
4,63.002222,1.507500,-106.731771,3.426736,123
3,57.661771,1.482188,-108.683125,1.100035,90
5,63.067674,1.100035,-111.523819,7.445278,154
6,63.261736,1.080729,-129.090833,7.653299,176
7,59.737639,0.319965,-155.473542,7.016875,216
8,60.148194,0.825590,-160.278958,10.196632,242
9,60.661910,0.429826,-167.830035,11.735139,278


# Save new config data to config file

In [5]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [1]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)